In [24]:
# The code was removed by Watson Studio for sharing.

a hospital supervisor needs to create a schedule for four nurses over a three-day period, subject to the following conditions:

Each day is divided into three 8-hour shifts.

Every day, each shift is assigned to a single nurse, and no nurse works more than one shift.

Each nurse is assigned to at least two shifts during the three-day period.

In [2]:
!pip install dse_do_utils

     |████████████████████████████████| 83 kB 4.3 MB/s  eta 0:00:01


In [3]:
from dse_do_utils import ScenarioManager
from docplex.mp.model import Model
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

from docplex import __version__
__version__

'2.22.213'

In [4]:
def continuous_var_series(df, mdl,**kargs):
    return pd.Series(mdl.continuous_var_list(df.index, **kargs), index = df.index)
def binary_var_series(df, mdl,**kargs):
    return pd.Series(mdl.binary_var_list(df.index, **kargs), index = df.index)
def integer_var_series(df, mdl,**kargs):
    '''Create a Series of integer dvar for each row in the DF. Most effective method. Best practice.
    Result can be assigned to a column of the df.
    Usage:
        df['xDVar'] = mdl.integer_var_series(df, name = 'xDVar')
    Args:
        mdl: CPLEX Model
        df: DataFrame
        **kargs: arguments passed to mdl.integer_var_list method. E.g. 'name'
        
    :returns: pandas.Series with integer dvars, index matches index of df
    '''
    #We are re-using the index from the DF index:
    return pd.Series(mdl.integer_var_list(df.index, **kargs), index = df.index)


class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)
    
    
def extract_solution(df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    df = df.copy()
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [dvar.solution_value for dvar in df[xDVarName]]
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df    

In [5]:
MODEL_NAME = 'NurseScheduling'
SCENARIO_NAME = 'Base_Scenario' 
mdl = Model(MODEL_NAME)

In [6]:
Number_Nurses = 4
Number_Days = 3
Number_Shifts = 3

Nurse = pd.DataFrame([])
Nurse["NurseID"] = ["Nurse" +str(i) for i in range(1,Number_Nurses+1)]

Days = pd.DataFrame([])
Days["DayID"] = ["Day" +str(i) for i in range(1,Number_Days+1)]

Shifts = pd.DataFrame([])
Shifts["ShiftsID"] = ["Shifts" +str(i) for i in range(1,Number_Shifts+1)]

In [7]:
Nurse_Shift = pd.merge(Days,Shifts, how = "cross" ).merge(Nurse, how = "cross").set_index(["DayID","ShiftsID","NurseID"], verify_integrity = True)

Nurse_Shift["X_Assigned"] = binary_var_series(Nurse_Shift, mdl, name = "X_Assigned" )

Nurse_Shift = Nurse_Shift.reset_index(drop = False)
Nurse_Shift

,DayID,ShiftsID,NurseID,X_Assigned
0,Day1,Shifts1,Nurse1,X_Assigned_Day1_Shifts1_Nurse1
1,Day1,Shifts1,Nurse2,X_Assigned_Day1_Shifts1_Nurse2
2,Day1,Shifts1,Nurse3,X_Assigned_Day1_Shifts1_Nurse3
3,Day1,Shifts1,Nurse4,X_Assigned_Day1_Shifts1_Nurse4
4,Day1,Shifts2,Nurse1,X_Assigned_Day1_Shifts2_Nurse1
5,Day1,Shifts2,Nurse2,X_Assigned_Day1_Shifts2_Nurse2
6,Day1,Shifts2,Nurse3,X_Assigned_Day1_Shifts2_Nurse3
7,Day1,Shifts2,Nurse4,X_Assigned_Day1_Shifts2_Nurse4
8,Day1,Shifts3,Nurse1,X_Assigned_Day1_Shifts3_Nurse1
9,Day1,Shifts3,Nurse2,X_Assigned_Day1_Shifts3_Nurse2


### each shift is assigned to a single nurse

In [8]:
Nurse_each_shift = Nurse_Shift[["DayID","ShiftsID","X_Assigned"]].groupby(["DayID","ShiftsID"]).agg(CplexSum(mdl)).reset_index(drop= False)

Nurse_each_shift

,DayID,ShiftsID,X_Assigned
0,Day1,Shifts1,X_Assigned_Day1_Shifts1_Nurse1+X_Assigned_Day1_Shifts1_Nurse2+X_Assigned_Day1_Shifts1_Nurse3+X_Assigned_Day1_Shifts1_Nurse4
1,Day1,Shifts2,X_Assigned_Day1_Shifts2_Nurse1+X_Assigned_Day1_Shifts2_Nurse2+X_Assigned_Day1_Shifts2_Nurse3+X_Assigned_Day1_Shifts2_Nurse4
2,Day1,Shifts3,X_Assigned_Day1_Shifts3_Nurse1+X_Assigned_Day1_Shifts3_Nurse2+X_Assigned_Day1_Shifts3_Nurse3+X_Assigned_Day1_Shifts3_Nurse4
3,Day2,Shifts1,X_Assigned_Day2_Shifts1_Nurse1+X_Assigned_Day2_Shifts1_Nurse2+X_Assigned_Day2_Shifts1_Nurse3+X_Assigned_Day2_Shifts1_Nurse4
4,Day2,Shifts2,X_Assigned_Day2_Shifts2_Nurse1+X_Assigned_Day2_Shifts2_Nurse2+X_Assigned_Day2_Shifts2_Nurse3+X_Assigned_Day2_Shifts2_Nurse4
5,Day2,Shifts3,X_Assigned_Day2_Shifts3_Nurse1+X_Assigned_Day2_Shifts3_Nurse2+X_Assigned_Day2_Shifts3_Nurse3+X_Assigned_Day2_Shifts3_Nurse4
6,Day3,Shifts1,X_Assigned_Day3_Shifts1_Nurse1+X_Assigned_Day3_Shifts1_Nurse2+X_Assigned_Day3_Shifts1_Nurse3+X_Assigned_Day3_Shifts1_Nurse4
7,Day3,Shifts2,X_Assigned_Day3_Shifts2_Nurse1+X_Assigned_Day3_Shifts2_Nurse2+X_Assigned_Day3_Shifts2_Nurse3+X_Assigned_Day3_Shifts2_Nurse4
8,Day3,Shifts3,X_Assigned_Day3_Shifts3_Nurse1+X_Assigned_Day3_Shifts3_Nurse2+X_Assigned_Day3_Shifts3_Nurse3+X_Assigned_Day3_Shifts3_Nurse4


In [9]:
for row in Nurse_each_shift.itertuples():
    mdl.add_constraint(row.X_Assigned  == 1)

## Each nurse works at most one shift per day.

In [11]:
Nurse_each_day = Nurse_Shift[["NurseID","DayID","X_Assigned"]].groupby(["NurseID","DayID"]).agg(CplexSum(mdl)).reset_index(drop= False)
Nurse_each_day

,NurseID,DayID,X_Assigned
0,Nurse1,Day1,X_Assigned_Day1_Shifts1_Nurse1+X_Assigned_Day1_Shifts2_Nurse1+X_Assigned_Day1_Shifts3_Nurse1
1,Nurse1,Day2,X_Assigned_Day2_Shifts1_Nurse1+X_Assigned_Day2_Shifts2_Nurse1+X_Assigned_Day2_Shifts3_Nurse1
2,Nurse1,Day3,X_Assigned_Day3_Shifts1_Nurse1+X_Assigned_Day3_Shifts2_Nurse1+X_Assigned_Day3_Shifts3_Nurse1
3,Nurse2,Day1,X_Assigned_Day1_Shifts1_Nurse2+X_Assigned_Day1_Shifts2_Nurse2+X_Assigned_Day1_Shifts3_Nurse2
4,Nurse2,Day2,X_Assigned_Day2_Shifts1_Nurse2+X_Assigned_Day2_Shifts2_Nurse2+X_Assigned_Day2_Shifts3_Nurse2
5,Nurse2,Day3,X_Assigned_Day3_Shifts1_Nurse2+X_Assigned_Day3_Shifts2_Nurse2+X_Assigned_Day3_Shifts3_Nurse2
6,Nurse3,Day1,X_Assigned_Day1_Shifts1_Nurse3+X_Assigned_Day1_Shifts2_Nurse3+X_Assigned_Day1_Shifts3_Nurse3
7,Nurse3,Day2,X_Assigned_Day2_Shifts1_Nurse3+X_Assigned_Day2_Shifts2_Nurse3+X_Assigned_Day2_Shifts3_Nurse3
8,Nurse3,Day3,X_Assigned_Day3_Shifts1_Nurse3+X_Assigned_Day3_Shifts2_Nurse3+X_Assigned_Day3_Shifts3_Nurse3
9,Nurse4,Day1,X_Assigned_Day1_Shifts1_Nurse4+X_Assigned_Day1_Shifts2_Nurse4+X_Assigned_Day1_Shifts3_Nurse4


In [12]:
for row in Nurse_each_day.itertuples():
    mdl.add_constraint(row.X_Assigned  <= 1)

## Each nurse is assigned to at least two shifts during the three-day period.

In [13]:
Nurse_day = Nurse_Shift[["NurseID","X_Assigned"]].groupby(["NurseID"]).agg(CplexSum(mdl)).reset_index(drop= False)
Nurse_day

,NurseID,X_Assigned
0,Nurse1,X_Assigned_Day1_Shifts1_Nurse1+X_Assigned_Day1_Shifts2_Nurse1+X_Assigned_Day1_Shifts3_Nurse1+X_Assigned_Day2_Shifts1_Nurse1+X_Assigned_Day2_Shifts2_Nurse1+X_Assigned_Day2_Shifts3_Nurse1+X_Assigned_Day3_Shifts1_Nurse1+X_Assigned_Day3_Shifts2_Nurse1+X_Assigned_Day3_Shifts3_Nurse1
1,Nurse2,X_Assigned_Day1_Shifts1_Nurse2+X_Assigned_Day1_Shifts2_Nurse2+X_Assigned_Day1_Shifts3_Nurse2+X_Assigned_Day2_Shifts1_Nurse2+X_Assigned_Day2_Shifts2_Nurse2+X_Assigned_Day2_Shifts3_Nurse2+X_Assigned_Day3_Shifts1_Nurse2+X_Assigned_Day3_Shifts2_Nurse2+X_Assigned_Day3_Shifts3_Nurse2
2,Nurse3,X_Assigned_Day1_Shifts1_Nurse3+X_Assigned_Day1_Shifts2_Nurse3+X_Assigned_Day1_Shifts3_Nurse3+X_Assigned_Day2_Shifts1_Nurse3+X_Assigned_Day2_Shifts2_Nurse3+X_Assigned_Day2_Shifts3_Nurse3+X_Assigned_Day3_Shifts1_Nurse3+X_Assigned_Day3_Shifts2_Nurse3+X_Assigned_Day3_Shifts3_Nurse3
3,Nurse4,X_Assigned_Day1_Shifts1_Nurse4+X_Assigned_Day1_Shifts2_Nurse4+X_Assigned_Day1_Shifts3_Nurse4+X_Assigned_Day2_Shifts1_Nurse4+X_Assigned_Day2_Shifts2_Nurse4+X_Assigned_Day2_Shifts3_Nurse4+X_Assigned_Day3_Shifts1_Nurse4+X_Assigned_Day3_Shifts2_Nurse4+X_Assigned_Day3_Shifts3_Nurse4


In [14]:
for row in Nurse_day.itertuples():
    mdl.add_constraint(row.X_Assigned  >= 2)

# Solve

In [16]:
mdl.print_information()

Model: NurseScheduling
 - number of variables: 36
   - binary=36, integer=0, continuous=0
 - number of constraints: 25
   - linear=25
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [17]:
mdl.solve()
mdl.report()

* model NurseScheduling solved with objective = 0.000


# Postprocessing

In [18]:
solution = extract_solution(Nurse_Shift, extract_dvar_names= ['X_Assigned'] ,drop=False)
solution.head()

,DayID,ShiftsID,NurseID,X_Assigned,X_Assigned_Solution
0,Day1,Shifts1,Nurse1,X_Assigned_Day1_Shifts1_Nurse1,0.0
1,Day1,Shifts1,Nurse2,X_Assigned_Day1_Shifts1_Nurse2,0.0
2,Day1,Shifts1,Nurse3,X_Assigned_Day1_Shifts1_Nurse3,1.0
3,Day1,Shifts1,Nurse4,X_Assigned_Day1_Shifts1_Nurse4,0.0
4,Day1,Shifts2,Nurse1,X_Assigned_Day1_Shifts2_Nurse1,1.0


In [19]:
solution = solution[solution["X_Assigned_Solution"]>0.1]
solution

,DayID,ShiftsID,NurseID,X_Assigned,X_Assigned_Solution
2,Day1,Shifts1,Nurse3,X_Assigned_Day1_Shifts1_Nurse3,1.0
4,Day1,Shifts2,Nurse1,X_Assigned_Day1_Shifts2_Nurse1,1.0
11,Day1,Shifts3,Nurse4,X_Assigned_Day1_Shifts3_Nurse4,1.0
14,Day2,Shifts1,Nurse3,X_Assigned_Day2_Shifts1_Nurse3,1.0
17,Day2,Shifts2,Nurse2,X_Assigned_Day2_Shifts2_Nurse2,1.0
20,Day2,Shifts3,Nurse1,X_Assigned_Day2_Shifts3_Nurse1,1.0
25,Day3,Shifts1,Nurse2,X_Assigned_Day3_Shifts1_Nurse2,1.0
30,Day3,Shifts2,Nurse3,X_Assigned_Day3_Shifts2_Nurse3,1.0
35,Day3,Shifts3,Nurse4,X_Assigned_Day3_Shifts3_Nurse4,1.0


In [22]:
Schedule = solution.pivot(index = "DayID", columns = "ShiftsID", values = "NurseID")
Schedule

ShiftsID,Shifts1,Shifts2,Shifts3
DayID,,,
Day1,Nurse3,Nurse1,Nurse4
Day2,Nurse3,Nurse2,Nurse1
Day3,Nurse2,Nurse3,Nurse4


# Save into scenario manager

In [25]:
sm = ScenarioManager(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, project =project )


InputTables={}

InputTables['Nurse']=Nurse
InputTables['Days']=Days
InputTables['Shifts']=Shifts

sm.inputs=InputTables



OutputTables={}
OutputTables['Schedule']=Schedule

sm.outputs=OutputTables

sm.write_data_into_scenario_s(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, inputs=InputTables, outputs=OutputTables)
